In [14]:
from ldc.lisa import orbits
from ldc.lisa.noise import get_noise_model
from astropy import units as un
import numpy as np
import matplotlib.pyplot as plt
import math
from spritz import utils, inference, masking, waveforms, sampling
import xarray as xr

In [2]:
dt=15
tobs=60*60*24*30*6
n_data=1036800

In [3]:
Param=np.array([-21.84367129,   7.74,       -20.62049506,  -0.56836753,   0.67456026,
  -0.18608149,   0.40864213,   1.89900672])


In [98]:
# Set orbit model
lisa_orbits = orbits.Orbits.type(dict({"nominal_arm_length":2.5e6*un.km,
                                        "initial_rotation":0*un.rad,
                                        "initial_position":0*un.rad,
                                        "orbit_type":"analytic"}))
pgb=utils.sample2waveform(Param)
# Set waveform model
my_gb = waveforms.MyFastGB(delta_t=dt, T=tobs, orbits=lisa_orbits) # in seconds

# Set noise model
freqs = np.fft.fftfreq(n_data)/dt
#set data 
Data_TF=my_gb.get_fd_tdixyz(template=pgb, oversample=4, tdi2=True,freqs=freqs)


noise_model = get_noise_model("spritz", freqs[freqs>0], wd=0)
    
# Set likelihood
Af = (Data_TF[2] - Data_TF[0]) / np.sqrt(2)
Ef = (Data_TF[0] - 2 * Data_TF[1] + Data_TF[2]) / np.sqrt(6)


def logliklihood(freqs,data,signal,s_n):
    df=1/(my_gb.T)
    snr2 = np.sum((data[:])*np.conjugate(signal[:]).real / s_n)
    hh = np.sum((np.conjugate(signal[:])*signal[:] / s_n))
    loglik = 4.0 * df * (snr2 - 0.5 * hh)


In [99]:
print(len(freqs))

1036800


In [66]:
import pandas as pd
print(type(my_gb))
print(type(Af))
af=xr.DataArray.to_numpy(Af)
ef=xr.DataArray.to_numpy(Ef)
print(type(af))
print(type(ef))

#xr.DataArray(array, coords=da.coords, dims=da.dims, attrs=da.attrs)
#fr1=pd.DataFrame(freqs, columns =['f'])
fr1=pd.Series(freqs) 
f=pd.DataFrame.to_xarray(fr1)
print(type(fr1))
print(type(f))
print(f)
#freqs_array=xr.Dataset.to_array(fr2)
#print(freqs_array)

<class 'spritz.waveforms.MyFastGB'>
<class 'xarray.core.dataarray.DataArray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'xarray.core.dataarray.DataArray'>
<xarray.DataArray (index: 1036800)>
array([ 0.00000000e+00,  6.43004115e-08,  1.28600823e-07, ...,
       -1.92901235e-07, -1.28600823e-07, -6.43004115e-08])
Coordinates:
  * index    (index) int64 0 1 2 3 4 ... 1036795 1036796 1036797 1036798 1036799


In [94]:
freqs_array=xr.DataArray(freqs,coords=[f])
#xr.DataArray(freqs, coords=da.coords, dims=da.dims, attrs=da.attrs)
print(freqs_array)
print(Af)

<xarray.DataArray (index: 1036800)>
array([ 0.00000000e+00,  6.43004115e-08,  1.28600823e-07, ...,
       -1.92901235e-07, -1.28600823e-07, -6.43004115e-08])
Coordinates:
  * index    (index) float64 0.0 6.43e-08 1.286e-07 ... -1.286e-07 -6.43e-08
<xarray.DataArray (f: 1036800)>
array([-1.58843123e-22-1.93115841e-22j, -1.58842538e-22-1.93116323e-22j,
       -1.58841952e-22-1.93116804e-22j, ...,
       -1.58844878e-22-1.93114397e-22j, -1.58844293e-22-1.93114879e-22j,
       -1.58843708e-22-1.93115360e-22j])
Coordinates:
  * f        (f) float64 -0.02559 -0.02559 -0.02559 ... 0.04107 0.04107 0.04107


In [100]:
loglike_cls = inference.GBLikelihood(freqs_array, [Af, Ef], 
                                         noise_model=noise_model, 
                                         signal_model=my_gb)


In [101]:
logvalue=loglike_cls.evaluate(Param)

ValueError: cannot reindex or align along dimension 'f' because the (pandas) index has duplicate values